# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [3]:
import pandas as pd
import numpy as np
from scipy import stats

In [4]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [7]:
data.shape

(4870, 65)

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

#1 What test is appropriate for this problem? Does CLT apply?
#2 What are the null and alternate hypotheses?

A: 

#1. The sample size is larger than 30, therefore CLT can be applied. Chi-square is appropriate for this problem since it's comparing two categories. 

#2. 
null hypothesis: there's no racial discrimination in the US job market.
alternative hypothesis: there's racial discrimination in the US job market.

In [9]:
w = data[data.race=='w']
b = data[data.race=='b']

In [25]:
# group by call
w_group = w.groupby(['call']).size()
b_group = b.groupby(['call']).size()
display(w_group)
display(b_group)

call
0.0    2200
1.0     235
dtype: int64

call
0.0    2278
1.0     157
dtype: int64

In [36]:
from scipy.stats import chi2_contingency
#build frequency table
frequency_table = pd.DataFrame.from_dict({'white':[w_group[0], w_group[1]], 'black':[b_group[0], b_group[1]]})

#display table
display(frequency_table)

#chisquare statistics
result = chi2_contingency(frequency_table)

#p-value
print ("p-value from chisquare is ", result[1])

,black,white
0,2278,2200
1,157,235


('p-value from chisquare is ', 4.9975783899632552e-05)


The result of p-value from chisquare analysis is approximated to 5E-5, which rejects the null hypothesis, stating that there's racial discrimination in the US job market.

Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [42]:
# bootstrap
def bootstrap_replicate_1d(data,func):
    return func(np.random.choice(data, size = len(data))), 

def draw_bs_reps(data, func, size = 1):
    
    #initializearray
    bs_replicate_w_0 = []
    bs_replicate_b_0 = []
    bs_replicate_w_1 = []
    bs_replicate_b_1 = []
    
    
    #generate replicates
    for i in range(size):
        data_temp = data.iloc[np.random.randint(0, len(data), size=size)]
        w = data_temp[data_temp.race=='w']
        b = data_temp[data_temp.race=='b']
        w_group = w.groupby(['call']).size()
        b_group = b.groupby(['call']).size()
        bs_replicate_w_0.append(w_group[0])
        bs_replicate_w_1.append(w_group[1])
        bs_replicate_b_0.append(b_group[0])
        bs_replicate_b_1.append(b_group[1])
        
    return func(bs_replicate_w_0), func(bs_replicate_w_1), func(bs_replicate_b_0), func(bs_replicate_b_1)


bs_replicate_w_0, bs_replicate_w_1, bs_replicate_b_0, bs_replicate_b_1 = draw_bs_reps(data, np.mean, 10000)


In [43]:
#build frequency table
bs_frequency_table = pd.DataFrame.from_dict({'white':[bs_replicate_w_0, bs_replicate_w_1], 'black':[bs_replicate_b_0, bs_replicate_b_1]})

#display table
display(bs_frequency_table)

#chisquare statistics
bs_result = chi2_contingency(bs_frequency_table)

#p-value
print ("p-value from bootstrap is ", bs_result[1])

,black,white
0,4677.4232,4517.1412
1,322.3939,483.0417


('p-value from bootstrap is ', 4.4789963112029905e-09)


The result from the bootstrap also showed that p-value is extremely small, indicating that there's racial discrimination in the US job market. 

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

#4.Write a story describing the statistical significance in the context or the original problem.

#5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?